In [1]:
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import urllib.parse
from urllib.parse import quote
import requests
import re
import time
import mimetypes
import tqdm

In [2]:
# Load the CEO data
df = pd.read_csv("ceo_data.csv")
df = df[df['Company'] != 'DXC Technology Company']
df.to_csv("ceo_data.csv", index=False)

In [3]:
base_url = "https://www.google.com/search?q={query}&tbm=isch&tbs=cdr:1,cd_min:{start_date},cd_max:{end_date}"

In [4]:
# Generate URLs for each CEO
search_urls = []
for _, row in df.iterrows():
    name = row["CEO"]
    firm = row["Company"]
    start_year = row["Year"]
    current_year = 2019  # Set this to the present year or desired end year
    
    for year in range(start_year, current_year + 1):
        query = quote(f"{name} {firm}")  # Encode the search query
        url = base_url.format(query=query, start_date=f"1/1/{year}", end_date=f"12/31/{year}")
        search_urls.append((name, year, url))
        break

In [5]:
# Save the URLs to a CSV file
output_filename = "ceo_image_search_urls.csv"
search_df = pd.DataFrame(search_urls, columns=["Name", "Year", "Search_URL"])
search_df
search_df.to_csv(output_filename, index=False)

print(f"Search URLs saved to {output_filename}")

Search URLs saved to ceo_image_search_urls.csv


In [6]:
## STEP 2: RUN GOOGLE IMAGE SEARCHES TO COLLECT PICTURE LINKS + ARTICLES THEY ARE IN

class GoogleImageScraper:
    def __init__(self):
        chrome_options = Options()
        # chrome_options.add_argument('--headless')
        self.driver = webdriver.Chrome(options=chrome_options)
        
    def hover_batch(self, elements, batch_size=5, delay=0.5):
        actions = ActionChains(self.driver)
        for i in range(0, len(elements), batch_size):
            batch = elements[i:i + batch_size]
            for el in batch:
                actions.move_to_element(el)
            actions.perform()
            time.sleep(delay)
                            
    def scrape_images(self, name, year, search_url, max_links=30):
        save_dir = os.path.join('pictures', name, str(year))
        os.makedirs(save_dir, exist_ok=True)
        
        try:
            self.driver.get(search_url)
            time.sleep(2)
            print(search_url)
            print(name, year)
            
            # Wait for search div and image elements
            search_div = WebDriverWait(self.driver, 10).until(
                EC.presence_of_element_located((By.ID, "search"))
            )
            time.sleep(2)

            print('hover start')        
            image_divs = search_div.find_elements(By.CSS_SELECTOR, 'div[jsname="qQjpJ"]')
            image_divs = image_divs[:max_links]  
            self.hover_batch(image_divs, batch_size=5, delay=0.5)
            print('hover done')        
               
            elements = WebDriverWait(search_div, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[jsname="qQjpJ"] h3 a'))
            )
            print(f"Found {len(elements)} potential image elements")
            time.sleep(2)

            image_urls = []
            for i, element in enumerate(elements):
                href = element.get_attribute('href')
                if href:
                    match = re.search(r'imgurl=([^&]+)', href)
                    match_story =  re.search(r'imgrefurl=([^&]+)', href)
                    if match and match_story:
                        image_url = urllib.parse.unquote(match.group(1))
                        story_url = urllib.parse.unquote(match_story.group(1))
                        print(f'element {i} has url {image_url}')                                                
                        image_urls.append([image_url, story_url])
                        if len(image_urls) >= max_links:
                            break
            
            # save image urls to csv
            image_urls_df = pd.DataFrame(image_urls, columns=['image_url', 'story_url'])
            image_urls_df.to_csv(os.path.join(save_dir, 'image_urls.csv'), index=False)            

        except Exception as e:
            print(f"Error processing {name} for year {year}: {e}")
                
    def close(self):
        self.driver.quit()

# Collect image articles/picture file links via selenium:

search_df = pd.read_csv("ceo_image_search_urls.csv").iloc[540:]

# test_df = search_df.sample(5) # run the search on subset 
# test_df

scraper = GoogleImageScraper()
for _, row in tqdm.tqdm(search_df.iterrows(), total=len(search_df), desc="Scraping"):

    try:
        files = os.listdir(f'pictures/{row["Name"]}/{row["Year"]}/')
    except:
        files = []
        
    if len([f for f in files if f.endswith('png') or f.endswith('jpg') or f.endswith('jpeg')]) >0:
        print('Skipping ',row['Name'], row['Year'])
        continue
    
    scraper.scrape_images(
        name=row["Name"],
        year=row["Year"],
        search_url=row["Search_URL"],
        max_links=4
    )
scraper.close()    

Scraping:   0%|                                         | 0/130 [00:00<?, ?it/s]

https://www.google.com/search?q=John%20Jumper%20Leidos%20Holdings%2C%20Inc.&tbm=isch&tbs=cdr:1,cd_min:1/1/2010,cd_max:12/31/2010
John Jumper 2010
hover start
hover done
Found 66 potential image elements


Scraping:   1%|▎                                | 1/130 [00:08<19:20,  9.00s/it]

element 0 has url https://techinquiry.org/logos/general_services_administration.svg
element 1 has url https://techinquiry.org/logos/defense_human_resources_activity.png
element 2 has url https://www.dnequipment.com/store/pc/catalog/Fred%20Cain%20AC-R45S.jpg
element 3 has url http://athpod.com/wp-content/uploads/2019/11/Chloe-Scott-Eli.jpg
https://www.google.com/search?q=John%20Jumper%20Leidos%20Holdings%2C%20Inc.&tbm=isch&tbs=cdr:1,cd_min:1/1/2011,cd_max:12/31/2011
John Jumper 2011
hover start
hover done
Found 100 potential image elements


Scraping:   2%|▌                                | 2/130 [00:17<18:24,  8.63s/it]

...

element 0 has url https://virtualizationreview.com/-/media/ECG/VirtualizationReview/Images/IntroImages2017/0717vrcm_F1Gelsinger.jpg
element 1 has url https://images.livemint.com/rf/Image-621x414/LiveMint/Period2/2017/10/28/Photos/Processed/PatGelsinger-klJB--621x414@LiveMint.JPG
element 2 has url https://media.bizj.us/view/img/10692410/gelsingerpat101217tj-10-web*1200xx1280-720-0-67.jpg
element 3 has url https://i.ytimg.com/vi/1S8pdLe8JCQ/sddefault.jpg
https://www.google.com/search?q=Pat%20Gelsinger%20VMware%2C%20Inc.&tbm=isch&tbs=cdr:1,cd_min:1/1/2018,cd_max:12/31/2018
Pat Gelsinger 2018
hover start
hover done
Found 100 potential image elements


Scraping:  99%|██████████████████████████████▊| 129/130 [18:36<00:08,  8.48s/it]

element 0 has url https://i.ytimg.com/vi/M_JedN8ww18/sddefault.jpg
element 1 has url https://i.ytimg.com/vi/FdIrbyMGvmw/mqdefault.jpg
element 2 has url https://s.wsj.net/public/resources/images/BN-DP279_0708vm_F_20140708155248.jpg
element 3 has url https://media.licdn.com/dms/image/v2/C4D03AQH_I7i1ET1yzA/profile-displayphoto-shrink_200_200/profile-displayphoto-shrink_200_200/0/1516256266518?e=2147483647&v=beta&t=_KiNHCY6hzreVwetL3kYYLr8gcckUo98Hkzxis0CTlk
https://www.google.com/search?q=Pat%20Gelsinger%20VMware%2C%20Inc.&tbm=isch&tbs=cdr:1,cd_min:1/1/2019,cd_max:12/31/2019
Pat Gelsinger 2019
hover start
hover done
Found 100 potential image elements


Scraping: 100%|███████████████████████████████| 130/130 [18:44<00:00,  8.65s/it]

element 0 has url https://i.insider.com/58bfd8e5dd08950d488b4a1a?width=799&format=jpeg
element 1 has url https://i.insider.com/5646139511231441008b4e55?width=933&format=jpeg
element 2 has url https://i.ytimg.com/vi/H2oonkOZkzY/maxresdefault.jpg
element 3 has url https://i.ytimg.com/vi/ieBwAUq9N-0/maxresdefault.jpg


In [8]:
import os
import mimetypes
import requests
import pandas as pd
import tqdm

search_df = pd.read_csv("ceo_image_search_urls.csv")

def download_images(image_urls, save_dir, limit=10):
    def get_file_extension(content_type):
        """Get file extension from content type"""
        extension = mimetypes.guess_extension(content_type)
        if extension:
            return extension
        # Fallback extensions
        extension_map = {
            'image/jpeg': '.jpg',
            'image/png': '.png',
            'image/webp': '.webp'
        }
        return extension_map.get(content_type, '.jpg')

    image_urls = image_urls[:limit]
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/91.0.4472.124'
    }

    print(f"Downloading {len(image_urls)} images")
    for i, url in enumerate(image_urls):
        try:
            img_response = requests.get(url, headers=headers, timeout=30)
            if img_response.status_code == 200:
                content_type = img_response.headers.get('content-type', 'image/jpeg')
                extension = get_file_extension(content_type)
                filename = f'pic{i+1}{extension}'
                filepath = os.path.join(save_dir, filename)
                with open(filepath, 'wb') as f:
                    f.write(img_response.content)
                print(f"Downloaded {filename}")
        except Exception as e:
            print(f"Error downloading image {i}: {e}")

for _, row in tqdm.tqdm(search_df.iterrows()):
    name = row['Name']
    year = str(row["Year"])
    save_dir = os.path.join('pictures', name, year)

    # Skip if images already exist
    if os.path.exists(save_dir):
        image_files = [f for f in os.listdir(save_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.webp'))]
        if image_files:
            print(f"Skipping {name} ({year}) - images already exist")
            continue

    csv_path = os.path.join(save_dir, 'image_urls.csv')
    
    # Skip if the CSV file does not exist
    if not os.path.exists(csv_path):
        print(f"Skipping {name} ({year}) - image_urls.csv not found")
        continue

    image_urls_df = pd.read_csv(csv_path)
    image_urls = image_urls_df['image_url'].tolist()
    download_images(image_urls, save_dir, limit=15)


0it [00:00, ?it/s]

Skipping Steve Jobs (2010) - images already exist
Skipping Tim Cook (2011) - images already exist
Skipping Tim Cook (2012) - images already exist
Skipping Tim Cook (2013) - images already exist
Skipping Tim Cook (2014) - images already exist
Skipping Tim Cook (2015) - images already exist
Skipping Tim Cook (2016) - images already exist
Skipping Tim Cook (2017) - images already exist
Skipping Tim Cook (2018) - images already exist
Skipping Tim Cook (2019) - images already exist
Skipping Steve Ballmer (2010) - images already exist
Skipping Steve Ballmer (2011) - images already exist
Skipping Steve Ballmer (2012) - images already exist
Skipping Steve Ballmer (2013) - images already exist
Skipping Satya Nadella (2014) - images already exist
Skipping Satya Nadella (2015) - images already exist
Skipping Satya Nadella (2016) - images already exist
Skipping Satya Nadella (2017) - images already exist
Skipping Satya Nadella (2018) - images already exist
Skipping Satya Nadella (2019) - images al

31it [00:00, 111.11it/s]

Skipping Scott A. McGregor (2011) - images already exist
Skipping Scott A. McGregor (2012) - images already exist
Skipping Scott A. McGregor (2013) - images already exist
Skipping Scott A. McGregor (2014) - images already exist
Skipping Scott A. McGregor (2015) - images already exist
Skipping Hock Tan (2016) - images already exist
Skipping Hock Tan (2017) - images already exist
Skipping Hock Tan (2018) - images already exist
Skipping Hock Tan (2019) - images already exist
Skipping Shantanu Narayen (2010) - images already exist
Skipping Shantanu Narayen (2011) - images already exist
Skipping Shantanu Narayen (2012) - images already exist
Skipping Shantanu Narayen (2013) - images already exist
Skipping Shantanu Narayen (2014) - images already exist
Skipping Shantanu Narayen (2015) - images already exist
Skipping Shantanu Narayen (2016) - images already exist
Skipping Shantanu Narayen (2017) - images already exist
Skipping Shantanu Narayen (2018) - images already exist
Skipping Shantanu N

106it [00:01, 75.49it/s]

...

Skipping Eddie Edwards (2019) - images already exist


621it [04:55,  1.05it/s]

Skipping Gary Smith (2011) - images already exist
Downloaded pic1.gif


670it [04:55,  2.27it/s]

Skipping Gary Smith (2013) - images already exist
Skipping Gary Smith (2014) - images already exist
Skipping Gary Smith (2015) - images already exist
Skipping Gary Smith (2016) - images already exist
Skipping Gary Smith (2017) - images already exist
Skipping Gary Smith (2018) - images already exist
Skipping Gary Smith (2019) - images already exist
Skipping Dominic Orr (2010) - images already exist
Skipping Dominic Orr (2011) - images already exist
Skipping Dominic Orr (2012) - images already exist
Skipping Dominic Orr (2013) - images already exist
Skipping Dominic Orr (2014) - images already exist
Skipping Dominic Orr (2015) - images already exist
Skipping Part of HPE (2016) - images already exist
Skipping Part of HPE (2017) - images already exist
Skipping Part of HPE (2018) - images already exist
Skipping Part of HPE (2019) - images already exist
Skipping Lane Bess (2010) - images already exist
Skipping Mark McLaughlin (2011) - images already exist
Skipping Mark McLaughlin (2012) - im